In [1]:
include("./ROIontology.jl")


ROIexp = load("./Data/ROIexp_FR.jld2", "ROIexp");
lfp = ROIexp[:lfp_data]
events = ROIexp[:behavioral_events]
trials = ROIexp[:trials]
sessions = ROIexp[:sessions]

include("./Links.jl")

WD = filter(x -> window_data_event(window_data(x)) ∈ dom(event_trial), collect(dom(window_data)))
WC = filter(x -> window_data_event(x) ∈ dom(event_trial), collect(codom(window_data)))


# WD = filter(x -> window_data_event(window_data(x)), collect(dom(window_data)))
# WC = filter(x -> window_data_event(x), collect(codom(window_data)))

1215-element Vector{Vector{Float64}}:
 [-0.09331660337249974, -0.09819589147917425, -0.09701250054912028, -0.10634067558245565, -0.1021405875302294, -0.09508570875457648, -0.10222455186734672, -0.09281840330007164, -0.04793675328783312, -0.057133355035141796  …  0.012000970506541482, 0.025241273428468974, 0.04294571134049535, 0.05226133557450283, 0.05718283311744984, 0.02164778581407128, -0.014923623848214692, 0.047510283889903915, 0.02752404050682769, 0.006529732011523056]
 [-0.13315295931214435, -0.13665682755355368, -0.11553105897029754, 0.01217419769535267, 0.12392939464261832, 0.16810780588327812, 0.13513692541079475, 0.15562960041545024, 0.14371666758303217, 0.1255655804435772  …  0.10221117842751364, 0.11151999093854242, 0.20395167324128263, 0.33968211693731504, 0.3435216408069801, 0.17291465716694754, -0.005134161633323996, -0.1969365304727736, -0.12453580032056083, -0.1391508127866966]
 [-0.01837882182201333, -0.05117392840454414, -0.05748236701138727, -0.04127012147527096, -0

In [2]:

using FourierAnalysis

using Impute: Impute, apply, DeclareMissings
using Missings: disallowmissing


function imputelfp(x)
    if window_data_region(x) == MOB
        thresh = 0.6
    else
        thresh = 0.4
    end
    x_clean = copy(x)
    x_clean[x.>thresh] .= NaN
    x_clean = apply(x_clean, DeclareMissings(; values = (NaN)))
    x_clean = Impute.interp(x_clean)
    return disallowmissings(x_clean)
end


# WT = event_trial∘window_data_event
# using JLD2 
# @save "TempData/event_lfp_windows_1_1.jld2"  WC
#split events into pre-peri-post with 512 sample pre and post windows
#events shorter than 512 are expanded
# wd_pre = GMap(x -> x[498:1010], WC)
wd_peri = GMap(x -> length(x) <= 2532 ? x[round(Int, length(x) / 2)-256:round(Int, length(x) / 2)+256] : x[1010:end-1010], WC)
# wd_peri = GMap(x -> x, WC)

# wd_post = GMap(x -> x[end-1010:end-498], WC)


# function isartifact(x::DataWindow)
#     if x.data.region==MOB
#         thresh=.75
#     else
#         thresh=.6
#     end
#     # if any(window_data(x).≥thresh)
#     if sum((window_data(x) .≥ thresh)) > (length(window_data(x)) * .01)
#         return true
#     else
#         return false
#     end
# end
# wd_peri=filter(!isartifact, win)

# #Freq Dom
# # pre_spec = GMap(x -> spectra(x, 256, 512), wd_pre.(WC))
peri_spec = GMap(x -> spectra(x, 256, 512), wd_peri.(WC))
# # post_spec = GMap(x -> spectra(x, 256, 512), wd_post.(WC)) 

using DataFrames, DataFramesMeta

event_spectra = DataFrame()

event_spectra.rat = map(x -> window_lfp(inv(window_data)(x)).rat.name, WC)
event_spectra.region = map(x -> window_data_region(x).name, WC)
event_spectra.behavior_type = map(x -> window_data_event(x).behavior.name, WC)
event_spectra.trial_type = map(x -> event_trial(window_data_event(x)).condition.name, WC)
event_spectra.agent_type = map(x -> agenttype(event_trial(window_data_event(x)).condition), WC)
specs = map(x -> spectra(x, 256, 512), WC)
event_spectra.respiratory = map(x -> mean(x, (3, 12) .* 256 ./ 1010.1), specs)
event_spectra.theta = map(x -> mean(x, (5, 10) .* 256 ./ 1010.1), specs)
event_spectra.beta = map(x -> mean(x, (15, 35) .* 256 ./ 1010.1), specs)
event_spectra.gamma_low = map(x -> mean(x, (50, 59) .* 256 ./ 1010.1), specs)
event_spectra.gamma_high = map(x -> mean(x, (70, 100) .* 256 ./ 1010.1), specs)


event_spectra = @subset(event_spectra, :trial_type .== "Free Roam", :region .∈ Ref(["MOB", "Amygdala", "Ca2"]))


,rat,region,behavior_type,trial_type,agent_type,respiratory,theta
,String7,String,String15,String,DataType,Float64,Float64
1,RRSD28,Ca2,Rearing,Free Roam,Robot,0.00480679,0.00552241
2,RRSD28,MOB,Immobility,Free Roam,Robot,0.0185517,0.015892
3,RRSD18,MOB,Grooming,Free Roam,Object,0.0136828,0.0121589
4,RRSD17,MOB,Rearing,Free Roam,Object,0.007176,0.00876942
5,RRSD17,Amygdala,Immobility,Free Roam,Object,0.00154076,0.00167873
6,RRSD17,Ca2,Immobility,Free Roam,Robot,0.00195673,0.00247963
7,RRSD17,MOB,Immobility,Free Roam,Rat,0.0124936,0.0149751
8,RRSD17,Amygdala,Grooming,Free Roam,Rat,0.00189254,0.00177685
9,RRSD28,Ca2,Immobility,Free Roam,Robot,0.00457848,0.0039664


In [117]:
ratidx = rand(findall(x -> x == Rat, collect(event_spectra.agent_type)), 50)
robotidx = rand(findall(x -> x == Robot, collect(event_spectra.agent_type)), 50)
objectidx = rand(findall(x -> x == Object, collect(event_spectra.agent_type)), 50)
nullidxs = vcat(ratidx, robotidx, objectidx)
esnull = copy(event_spectra)
esnull.behavior_type = fill("null", length(esnull.behavior_type))

event_spectra=vcat(event_spectra, esnull[nullidxs,:])


,rat,region,behavior_type,trial_type,agent_type,respiratory,theta
,String7,String,String,String,DataType,Float64,Float64
1,RRSD28,Ca2,Rearing,Free Roam,Robot,0.00480679,0.00552241
2,RRSD28,MOB,Immobility,Free Roam,Robot,0.0185517,0.015892
3,RRSD18,MOB,Grooming,Free Roam,Object,0.0136828,0.0121589
4,RRSD17,MOB,Rearing,Free Roam,Object,0.007176,0.00876942
5,RRSD17,Amygdala,Immobility,Free Roam,Object,0.00154076,0.00167873
6,RRSD17,Ca2,Immobility,Free Roam,Robot,0.00195673,0.00247963
7,RRSD17,MOB,Immobility,Free Roam,Rat,0.0124936,0.0149751
8,RRSD17,Amygdala,Grooming,Free Roam,Rat,0.00189254,0.00177685
9,RRSD28,Ca2,Immobility,Free Roam,Robot,0.00457848,0.0039664


In [118]:
using MixedModels, ProgressMeter
ProgressMeter.ijulia_behavior(:append)

IJuliaAppend::IJuliaBehavior = 2

MOB

Grooming

In [119]:
#low gamma
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .== "Grooming")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_low ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0024 | 0.0009 |  2.75 | 0.0059 | 0.0015 |
| agent_type: Object | -0.0000 | 0.0002 | -0.07 | 0.9448 |        |
| agent_type: Robot  |  0.0001 | 0.0001 |  1.01 | 0.3109 |        |
| Residual           |  0.0006 |        |       |        |        |


In [82]:
#beta
fm = @formula(beta ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0013 | 0.0003 |  4.88 | <1e-05 | 0.0004 |
| agent_type: Object | -0.0001 | 0.0001 | -0.64 | 0.5190 |        |
| agent_type: Robot  | -0.0001 | 0.0001 | -1.37 | 0.1692 |        |
| Residual           |  0.0002 |        |       |        |        |


In [134]:
using GLM
fm = @formula(beta ~ 0 + agent_type)
fm1 = fit(LinearModel, fm, hyposub)# contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

beta ~ 0 + agent_type

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                          Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
agent_type: Object  0.00094772   8.81089e-5  10.76    <1e-17  0.000772893  0.00112255
agent_type: Rat     0.000702202  5.51759e-5  12.73    <1e-21  0.000592721  0.000811683
agent_type: Robot   0.00085841   7.07707e-5  12.13    <1e-20  0.000717986  0.000998835
──────────────────────────────────────────────────────────────────────────────────────

In [131]:
DataFrame(coeftable(fm1))

,Name,Coef.,Std. Error,z,Pr(>|z|)
,String,Float64,Float64,Float64,Float64
1,(Intercept),0.000737807,0.000184066,4.00838,6.11358e-5
2,agent_type: Object,-0.000130321,4.35621e-5,-2.99162,0.00277499
3,agent_type: Robot,-5.87892e-5,3.47817e-5,-1.69023,0.0909837


In [130]:
#high gamma
fm = @formula(gamma_high ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0007 | 0.0002 |  4.01 | <1e-04 | 0.0003 |
| agent_type: Object | -0.0001 | 0.0000 | -2.99 | 0.0028 |        |
| agent_type: Robot  | -0.0001 | 0.0000 | -1.69 | 0.0910 |        |
| Residual           |  0.0001 |        |       |        |        |


Rearing

In [149]:
#high gamma
hyposub = @subset(event_spectra, :region .== "Amygdala", :behavior_type .== "Immobility")
function afun(atype)
    if atype == Rat
        return "Rat"
    elseif atype == Object
        return "Object"
    elseif atype == Robot
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(theta ~ 0 + agent_type + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub) #contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |   Est. |     SE |     z |      p |  σ_rat |
|:------------------ | ------:| ------:| -----:| ------:| ------:|
| agent_type: Object | 0.0014 | 0.0001 | 10.20 | <1e-23 |        |
| agent_type: Rat    | 0.0015 | 0.0002 |  9.63 | <1e-21 |        |
| agent_type: Robot  | 0.0014 | 0.0001 | 10.15 | <1e-23 |        |
| (Intercept)        |        |        |       |        | 0.0002 |
| Residual           | 0.0003 |        |       |        |        |


In [85]:
#theta
fm = @formula(theta ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0250 | 0.0055 |  4.54 | <1e-05 | 0.0094 |
| agent_type: Object | -0.0019 | 0.0016 | -1.16 | 0.2456 |        |
| agent_type: Robot  | -0.0009 | 0.0012 | -0.73 | 0.4632 |        |
| Residual           |  0.0049 |        |       |        |        |


Immobitlity

In [86]:
#low gamma
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .== "Immobility")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_low ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |   Est. |     SE |    z |      p |  σ_rat |
|:------------------ | ------:| ------:| ----:| ------:| ------:|
| (Intercept)        | 0.0007 | 0.0003 | 2.28 | 0.0227 | 0.0004 |
| agent_type: Object | 0.0001 | 0.0002 | 0.41 | 0.6818 |        |
| agent_type: Robot  | 0.0002 | 0.0002 | 0.85 | 0.3938 |        |
| Residual           | 0.0007 |        |      |        |        |


CA

Grooming

In [142]:
#theta
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "Amygdala", :behavior_type .== "Grooming")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(theta ~ 0 + agent_type + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub)#, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |   Est. |     SE |     z |      p |  σ_rat |
|:------------------ | ------:| ------:| -----:| ------:| ------:|
| agent_type: Object | 0.0012 | 0.0001 | 10.72 | <1e-26 |        |
| agent_type: Rat    | 0.0013 | 0.0001 | 11.92 | <1e-32 |        |
| agent_type: Robot  | 0.0012 | 0.0001 | 10.69 | <1e-25 |        |
| (Intercept)        |        |        |       |        | 0.0002 |
| Residual           | 0.0003 |        |       |        |        |


In [88]:
#high gamma
fm = @formula(gamma_high ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0000 | 0.0000 |  8.95 | <1e-18 | 0.0000 |
| agent_type: Object |  0.0000 | 0.0000 |  1.06 | 0.2877 |        |
| agent_type: Robot  | -0.0000 | 0.0000 | -1.14 | 0.2524 |        |
| Residual           |  0.0000 |        |       |        |        |


Rearing

In [89]:
#high gamma
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "Ca2", :behavior_type .== "Rearing")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_high ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |   Est. |     SE |    z |      p |  σ_rat |
|:------------------ | ------:| ------:| ----:| ------:| ------:|
| (Intercept)        | 0.0000 | 0.0000 | 3.99 | <1e-04 | 0.0000 |
| agent_type: Object | 0.0000 | 0.0000 | 3.08 | 0.0021 |        |
| agent_type: Robot  | 0.0000 | 0.0000 | 2.95 | 0.0032 |        |
| Residual           | 0.0000 |        |      |        |        |


Immobility

In [90]:
#high gamma
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "Ca2", :behavior_type .== "Immobility")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_high ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0000 | 0.0000 | 10.94 | <1e-27 | 0.0000 |
| agent_type: Object | -0.0000 | 0.0000 | -0.08 | 0.9349 |        |
| agent_type: Robot  | -0.0000 | 0.0000 | -0.81 | 0.4196 |        |
| Residual           |  0.0000 |        |       |        |        |


Amygdala

Grooming

In [91]:
#low gamma
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "Amygdala", :behavior_type .== "Grooming")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_low ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |   Est. |     SE |    z |      p |  σ_rat |
|:------------------ | ------:| ------:| ----:| ------:| ------:|
| (Intercept)        | 0.0001 | 0.0000 | 3.43 | 0.0006 | 0.0001 |
| agent_type: Object | 0.0000 | 0.0000 | 0.79 | 0.4272 |        |
| agent_type: Robot  | 0.0000 | 0.0000 | 0.50 | 0.6176 |        |
| Residual           | 0.0000 |        |      |        |        |


In [92]:
#theta
fm = @formula(theta ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0013 | 0.0001 | 12.93 | <1e-37 | 0.0001 |
| agent_type: Object | -0.0002 | 0.0001 | -2.07 | 0.0384 |        |
| agent_type: Robot  | -0.0001 | 0.0001 | -2.06 | 0.0396 |        |
| Residual           |  0.0003 |        |       |        |        |


Rearing

In [93]:
#theta 
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "Amygdala", :behavior_type .== "Rearing")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(theta ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0015 | 0.0002 |  8.63 | <1e-17 | 0.0003 |
| agent_type: Object |  0.0001 | 0.0001 |  1.34 | 0.1810 |        |
| agent_type: Robot  | -0.0002 | 0.0001 | -2.78 | 0.0055 |        |
| Residual           |  0.0003 |        |       |        |        |


Immobility

In [94]:
#low gamma
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "Amygdala", :behavior_type .== "Immobility")
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_low ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0001 | 0.0000 |  3.81 | 0.0001 | 0.0000 |
| agent_type: Object | -0.0000 | 0.0000 | -0.40 | 0.6913 |        |
| agent_type: Robot  | -0.0000 | 0.0000 | -0.13 | 0.8979 |        |
| Residual           |  0.0000 |        |       |        |        |


In [95]:
#high gamma
fm = @formula(gamma_high ~ 1 + agent_type * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0000 | 0.0000 |  3.21 | 0.0013 | 0.0000 |
| agent_type: Object | -0.0000 | 0.0000 | -1.16 | 0.2468 |        |
| agent_type: Robot  | -0.0000 | 0.0000 | -0.15 | 0.8810 |        |
| Residual           |  0.0000 |        |       |        |        |
